In [1]:

import os
import shutil
import unittest

from arc.common import ARC_PATH, almost_equal_lists, read_yaml_file
from arc.job.adapters.pyscf import PYSCFAdapter
from arc.level import Level
from arc.species import ARCSpecies
from arc.species.vectors import calculate_distance, calculate_angle, calculate_dihedral_angle
from arc.utils.wip import work_in_progress



/home/calvin/mambaforge/envs/arc_env/lib/python3.7/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:

etoh_xyz = """ C       -0.93790674    0.28066443    0.10572942
                C        0.35659906   -0.44954997    0.05020174
                O        0.36626530   -1.59397979   -0.38012632
                H       -1.68923915   -0.33332195    0.61329151
                H       -0.85532021    1.23909997    0.62578027
                H       -1.30704889    0.46001151   -0.90948878
                H        0.76281007   -0.50036590    1.06483009
                H        1.04287051    0.12137561   -0.58236096
                H        1.27820018   -1.93031032   -0.35203473"""

In [3]:
        pyscf_buildinput = PYSCFAdapter(execution_type='incore',
                                    job_type='opt',
                                    project='test_2',
                                    project_directory=os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter'),
                                        level=Level(software='pyscf',
                                        method='b3lyp',
                                        basis='def2-TZVP',),
                                    species=[ARCSpecies(label='EtOH', smiles='CCO', xyz=etoh_xyz)],)


In [4]:
pyscf_buildinput.write_input_file()


In [5]:
import pandas as pd
# use pandas to read the yml file
df = pd.read_csv(os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter','calcs','Species','EtOH','opt_a10','input.yml'))

In [6]:
# read the yml file as a dictionary
input_dict = read_yaml_file(os.path.join(ARC_PATH, 'arc', 'testing', 'test_PYSCFAdapter','calcs','Species','EtOH','opt_a10','input.yml'))
input_dict

{'basis': 'def2-tzvp',
 'charge': 0,
 'is_ts': 'False',
 'job_type': 'opt',
 'restricted': 'False',
 'spin': 0,
 'xc_func': 'b3lyp',
 'xyz': 'C      -0.93790674    0.28066443    0.10572942\nC       0.35659906   -0.44954997    0.05020174\nO       0.36626530   -1.59397979   -0.38012632\nH      -1.68923915   -0.33332195    0.61329151\nH      -0.85532021    1.23909997    0.62578027\nH      -1.30704889    0.46001151   -0.90948878\nH       0.76281007   -0.50036590    1.06483009\nH       1.04287051    0.12137561   -0.58236096\nH       1.27820018   -1.93031032   -0.35203473'}

In [1]:
#!/usr/bin/env python3
# encoding: utf-8

"""
A standalone script to run PySCF with the xyz data outputted from ARC.
Should be run under the pyscf_env.
"""

# Parse the input.yml file

import os
import sys
import yaml
import argparse
import pyscf
from pyscf import gto, scf

class PYSCFScript:
    
    def __init__(self, input_file):
        self.input_file = input_file
        self.input_dict = self.read_input_file()
        self.job_type = self.get_job_type()
        self.mol = self.get_mol()
        self.method = self.get_method()
        self.basis = self.get_basis()
        self.charge = self.get_charge()
        self.spin = self.get_spin()
        self.restricted = self.get_restricted()
        self.run()
    def read_input_file(self):
        with open(self.input_file, 'r') as f:
            input_dict = yaml.load(f)
        return input_dict
    def get_job_type(self):
        job_type = self.input_dict['job_type']
        return job_type
    def get_mol(self):
        mol = self.input_dict['molecule']
        return mol
    def get_method(self):
        method = self.input_dict['xc_func']
        return method
    def get_basis(self):
        basis = self.input_dict['basis']
        return basis
    def get_charge(self):
        charge = self.input_dict['charge']
        return charge
    def get_spin(self):
        spin = self.input_dict['spin']
        return spin
    def get_restricted(self):
        restricted = self.input_dict['restricted']
        return restricted
    def run(self):

        mol = gto.Mole()
        mol.atom = self.mol
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        
        if restricted:
            mf = dft.RKS(mol)
        else:
            mf = dft.UKS(mol)
        mf.xc = self.method
        
        # TODO: Check if TS
        if self.is_ts and self.job_type == 'opt':
            from pyscf.qsdopt.qsd_optimizer import QSD
            optimizer = QSD(mf, stationary_point='TS')
            

            # hess_update_freq: Frequency for numerical reevaluation of hessian. = 0 evaluates the numerical hessian in the first iteration and is updated with an BFGS rule, unless approaching a trap region, where it is reevaluated. (Default: 0)

            # numhess_method: Method for evaluating numerical hessian. Forward and central differences are available with “forward” and “central”, respectively. (Default: “forward”)

            # max_iter: Maximum number of optimization steps. (Default: 100)

            # step: Maximum norm between two optimization steps. (Default: 0.1)

            # hmin: Minimum distance between current geometry and stationary point of the quadratic form to consider convergence reached. (Default: 1e-6)

            # gthres: Gradient norm to consider convergence. (Default: 1e-5)

            optimizer.kernel(hess_update_freq=0, numhess_method='forward', max_iter=100, step=0.1, hmin=1e-6, gthres=1e-5)
        
        if self.job_type == 'opt' and not self.is_ts:
            from pyscf.geomopt.geometric_solver import optimize
            conv_params = { # These are the default settings
                                'convergence_energy': 1e-6,  # Eh
                                'convergence_grms': 3e-4,    # Eh/Bohr
                                'convergence_gmax': 4.5e-4,  # Eh/Bohr
                                'convergence_drms': 1.2e-3,  # Angstrom
                                'convergence_dmax': 1.8e-3,  # Angstrom
                            }
            mol_eq = optimize(mf, maxsteps=100, **conv_params) # TODO: Need to define maxsteps as a TRSH parameter
            mf.kernel()
            print('SCF energy of {0} is {1}.'.format(self.method, mf.e_tot))
            
        if self.job_type == 'freq':
            #npip install git+https://github.com/pyscf/properties
            from pyscf.prop.freq import rks            
            w, modes = rks.Freq(mf).kernel()            
            print('Frequencies (cm-1):')
        
        # if self.job_type == 'scan':
        #     #https://github.com/pyscf/pyscf/blob/14d88828cd1f18f1e5358da1445355bde55322a1/examples/scf/30-scan_pes.py#L16
        #     # Requires manual development for the code. Not feasible at the moment.
        #     scan_results = []
        #     for value in np.arange(self.start, self.end, self.step_size):
        #         upd
        if self.job_type == 'sp':
            mf.kernel()
            print('The single-point DFT energy is:', energy)
            
        print('SCF energy of {0} is {1}.'.format(self.method, mf.e_tot))

ModuleNotFoundError: No module named 'pyscf'